In [1]:
from pykrx import stock
import pandas as pd
import FinanceDataReader as fdr
import numpy as np

In [2]:
df = pd.read_csv('C:/Users/Administrator/Desktop/미래에셋 공모전/data/전종목정보.csv')

#불러오면 종목코드가 int로 바뀌는 현상이 나타나서 다시 Zerofill 진행
df['Code'] = df['Code'].apply(lambda x : str(x).zfill(6))

In [3]:
#연도별로 데이터 가져오기
stock_2022 = stock.get_market_cap("20221229")
stock_2021 = stock.get_market_cap("20211230")
stock_2020 = stock.get_market_cap("20201230")
stock_2019 = stock.get_market_cap("20191230")
stock_2018 = stock.get_market_cap("20181228")
stock_2017 = stock.get_market_cap("20171228")

#종목코드 컬럼 생성
stock_2022['Code'] = stock_2022.index
stock_2021['Code'] = stock_2021.index
stock_2020['Code'] = stock_2020.index
stock_2019['Code'] = stock_2019.index
stock_2018['Code'] = stock_2018.index
stock_2017['Code'] = stock_2017.index

#연도 컬럼 생성
stock_2022['연도']= 2022
stock_2021['연도']= 2021
stock_2020['연도']= 2020
stock_2019['연도']= 2019
stock_2018['연도']= 2018
stock_2017['연도']= 2017

#사용되는 종목 필터링
stock_2022 = stock_2022[stock_2022["Code"].isin(df.Code)]
stock_2021 = stock_2021[stock_2021["Code"].isin(df.Code)]
stock_2020 = stock_2020[stock_2020["Code"].isin(df.Code)]
stock_2019 = stock_2019[stock_2019["Code"].isin(df.Code)]
stock_2018 = stock_2018[stock_2018["Code"].isin(df.Code)]
stock_2017 = stock_2017[stock_2017["Code"].isin(df.Code)]

#필요한 열 추출
stock_2022 = stock_2022.loc[:,['연도', 'Code','시가총액','상장주식수']]
stock_2021 = stock_2021.loc[:,['연도', 'Code','시가총액','상장주식수']]
stock_2020 = stock_2020.loc[:,['연도', 'Code','시가총액','상장주식수']]
stock_2019 = stock_2019.loc[:,['연도', 'Code','시가총액','상장주식수']]
stock_2018 = stock_2018.loc[:,['연도', 'Code','시가총액','상장주식수']]
stock_2017 = stock_2017.loc[:,['연도', 'Code','시가총액','상장주식수']]

#인덱스 초기화
stock_2022 = stock_2022.reset_index(drop=True)
stock_2021 = stock_2021.reset_index(drop=True)
stock_2020 = stock_2020.reset_index(drop=True)
stock_2019 = stock_2019.reset_index(drop=True)
stock_2018 = stock_2018.reset_index(drop=True)
stock_2017 = stock_2017.reset_index(drop=True)

stock = pd.concat([stock_2022,stock_2021,stock_2020,stock_2019,stock_2018,stock_2017], ignore_index=True)

# 주가 수집

In [4]:
#사용되는 종목 리스트 가져오기
df = pd.read_csv('C:/Users/Administrator/Desktop/미래에셋 공모전/data/재무제표_전처리.csv')

date = ['20221229', "20211230", "20201230", '20191230', '20181228', '20171228']

stocks_all = fdr.StockListing('KRX-DESC')
stocks_sec = stocks_all[stocks_all["Name"].isin(df.기업.unique())].loc[:,['Code','Name']]

stock_price = pd.DataFrame()

date = ['20221229', "20211230", "20201230", '20191230', '20181228', '20171228']
종가 = []
Code = []
연도 = []

for ix, row in stocks_sec.iterrows():
    code, name = row['Code'], row['Name']
    
    for idx, year in enumerate(date):
        # 개별 종목의 가격 가져오기
        try:
            dat = fdr.DataReader(code, year, year)

            종가.append(dat['Close'].iloc[0])
            Code.append(code)
            연도.append(int(year[0:4]))
        
        except:
            종가.append(np.nan)
            Code.append(code)
            연도.append(int(year[0:4]))

stock_price['종가'] = 종가
stock_price['Code'] = Code
stock_price['연도'] = 연도

In [5]:
stock_price

,종가,Code,연도
0,2095.0,060310,2022
1,3325.0,060310,2021
2,2165.0,060310,2020
3,2820.0,060310,2019
4,2215.0,060310,2018
...,...,...,...
9361,9850.0,238490,2021
9362,12800.0,238490,2020
9363,13625.0,238490,2019
9364,4749.0,238490,2018


In [6]:
stock

,연도,Code,시가총액,상장주식수
0,2022,005930,330128975015000,5969782550
1,2022,207940,58433854000000,71174000
2,2022,000660,54600177375000,728002365
3,2022,051910,42355405800000,70592343
4,2022,006400,40639837230000,68764530
...,...,...,...,...
9163,2017,001140,15780011520,1682304
9164,2017,227950,14949000000,7550000
9165,2017,173130,12018383100,2967502
9166,2017,215090,10972500000,5500000


In [7]:
merge = pd.merge(stock,stock_price, how='left', left_on = ['Code','연도'], right_on = ['Code','연도'])

In [8]:
merge

,연도,Code,시가총액,상장주식수,종가
0,2022,005930,330128975015000,5969782550,55300.0
1,2022,207940,58433854000000,71174000,821000.0
2,2022,000660,54600177375000,728002365,75000.0
3,2022,051910,42355405800000,70592343,600000.0
4,2022,006400,40639837230000,68764530,591000.0
...,...,...,...,...,...
9163,2017,001140,15780011520,1682304,9380.0
9164,2017,227950,14949000000,7550000,2047.0
9165,2017,173130,12018383100,2967502,1376.0
9166,2017,215090,10972500000,5500000,2591.0


In [9]:
#데이터 concat 후 저장

merge.to_csv('연도별시가총액및주가상장주식수.csv', index=False, encoding='UTF-8-sig')

In [10]:
merge.loc[merge['Code']=='051900', ['연도','종가','Code']]

,연도,종가,Code
11,2022,722000.0,051900
1570,2021,1097000.0,051900
3128,2020,1620000.0,051900
4689,2019,1261000.0,051900
6245,2018,1101000.0,051900
7744,2017,1189000.0,051900


In [11]:
merge.loc[merge['Code']=='005930', ['연도','종가','Code']]

,연도,종가,Code
0,2022,55300.0,005930
1561,2021,78300.0,005930
3122,2020,81000.0,005930
4683,2019,55800.0,005930
6238,2018,38700.0,005930
7736,2017,50960.0,005930
